In [7]:
from generatexyz import Generatexyz
import numpy as np
from gan import load_weight
from gan import GenAdvNetwork
import tensorflow as tf
from utils import random_generator
from reformatToXyz import format_xyz_samples
import os
import pathlib
from reformatToXyz import MapCoulomb1dToXYZ
from formatData import loadData
from sklearn.model_selection import train_test_split
from coulombToTraj import getFlattenedXyz

In [2]:
benzene_xyz = [0.000000000000000000e+00,1.397000000000000020e+00,0.000000000000000000e+00,
               1.209799999999999986e+00,6.985000000000000098e-01,0.000000000000000000e+00,
               1.209799999999999986e+00,-6.985000000000000098e-01,0.000000000000000000e+00,
               0.000000000000000000e+00,-1.397000000000000020e+00,0.000000000000000000e+00,
               -1.209799999999999986e+00,-6.985000000000000098e-01,0.000000000000000000e+00,
               -1.209799999999999986e+00,6.985000000000000098e-01,0.000000000000000000e+00,
               0.000000000000000000e+00,2.480999999999999872e+00,0.000000000000000000e+00,
               2.148600000000000065e+00,1.240499999999999936e+00,0.000000000000000000e+00,
               2.148600000000000065e+00,-1.240499999999999936e+00,0.000000000000000000e+00,
               0.000000000000000000e+00,-2.480999999999999872e+00,0.000000000000000000e+00,
               -2.148600000000000065e+00,-1.240499999999999936e+00,0.000000000000000000e+00,
               -2.148600000000000065e+00,1.240499999999999936e+00,0.000000000000000000e+00]
benzene_xyz = np.array(benzene_xyz)
gen_obj = Generatexyz(1000000, 1000, 12)
gen_obj.generate_samples(benzene_xyz)
gen_obj.sorting_by_coulomb_matrix()

Current Batch:  0
Current Batch:  1
Current Batch:  2
Current Batch:  3
Current Batch:  4
Current Batch:  5
Current Batch:  6
Current Batch:  7
Current Batch:  8
Current Batch:  9
Current Batch:  10
Current Batch:  11
Current Batch:  12
Current Batch:  13
Current Batch:  14
Current Batch:  15
Current Batch:  16
Current Batch:  17
Current Batch:  18
Current Batch:  19
Current Batch:  20
Current Batch:  21
Current Batch:  22
Current Batch:  23
Current Batch:  24
Current Batch:  25
Current Batch:  26
Current Batch:  27
Current Batch:  28
Current Batch:  29
Current Batch:  30
Current Batch:  31
Current Batch:  32
Current Batch:  33
Current Batch:  34
Current Batch:  35
Current Batch:  36
Current Batch:  37
Current Batch:  38
Current Batch:  39
Current Batch:  40
Current Batch:  41
Current Batch:  42
Current Batch:  43
Current Batch:  44
Current Batch:  45
Current Batch:  46
Current Batch:  47
Current Batch:  48
Current Batch:  49
Current Batch:  50
Current Batch:  51
Current Batch:  52
Cur

In [3]:
latent_dim_ = 78
batch_size_ = 32
weights_path = "/home/panthibivek/thesis/GAN_pkg/runs/train/exp2/weights/"
GanModel = GenAdvNetwork(latent_dim=latent_dim_, batch_size=batch_size_)
GanModel.compile(
    generator_opt=tf.keras.optimizers.Adam(learning_rate=0.001),
    discriminator_opt=tf.keras.optimizers.Adam(learning_rate=0.001),
    disc_loss=tf.keras.losses.BinaryCrossentropy(),
    gen_loss=tf.keras.losses.MAE
)

random_data = random_generator((1000, latent_dim_))

GanModel.generator(random_data)
GanModel.load_weights(weights_path) 
generated_output = GanModel.generator(random_data)
arr = generated_output.numpy()
arr_new = arr.reshape((1000,78))

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 13)                1027      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 13)                0         
                                                                 
 reshape (Reshape)           (None, 13, 1)             0         
                                                                 
 conv1d_transpose (Conv1DTra  (None, 39, 128)          640       
 nspose)                                                         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 39, 128)           0         
                                                                 
 batch_normalization (BatchN  (None, 39, 128)          512       
 ormalization)                                           

2023-03-26 21:32:17.749619: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-26 21:32:17.749689: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-26 21:32:17.810754: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-26 21:32:17.810818: I tensorflow/compiler/xla/stream_executor/rocm/rocm_gpu_executor.cc:843] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-26 21:32:17.810852: I tensorflow/compiler/xla/stream_executo

In [4]:
parent_dir_coulomb1D_to_xyz = pathlib.Path(os.path.abspath(os.path.dirname('__file__')) + "/data/MolueculesMappedFromSampleSpace")
parent_dir_coulomb1D_to_xyz.mkdir(parents=True, exist_ok=True)

mapping_obj = MapCoulomb1dToXYZ(abs_path_xyz_dirname=str(parent_dir_coulomb1D_to_xyz), 
                                sorted_xyz_file=gen_obj.sorted_traj_dirname, sorted_coulomb1D_file=gen_obj.sorted_coulomb1D_dirname)
mapping_obj.generateXYZ(coulomb1D_arr=arr_new, batch_size=1000)

364.1258369201182
898.2434892502838
331.65802601716143
793.1746479565911
406.9608908787692
358.79765060697395
346.81009559908244
313.1071530917914
358.9358327917927
397.12625410212763
1590.9556981392511
316.8592737791924
263.388385478039
358.7091939653858
280.9614424503889
742.6353702549906
352.9548395918588
407.66319956366107
276.05123943348684
334.26261816088834
402.57328589715155
365.522786221657
318.37958800480874
1257.508401340176
233.84913903939932
313.3466825208594
368.35378279972616
357.98730195946337
423.930567363478
348.1943684361733
355.60435189763814
260.9237135961553
366.0232080807842
436.63335679931083
446.6657388282432
427.0417070047289
444.92755928834003
608.7131832035442
275.29565216710307
352.6742420411577
435.91364407936663
459.40257338562924
455.19553109974595
402.7562082103223
344.93685384547445
258.2126805718736
307.81201067256063
1641.3695624311529
296.3744757983253
332.20729397370604
344.2791591441821
338.79312067454003
240.08873721484989
283.3601862333774
490.4

In [1]:
import numpy as np
import os
import pathlib
from reformatToXyz import MapCoulomb1dToXYZ
from formatData import loadData
from coulombToTraj import getFlattenedXyz

xyz_files_dir_name = "/home/panthibivek/thesis/GAN_pkg/data/AllMolecules/"
xyz_traj_filename = "/home/panthibivek/thesis/GAN_pkg/data/traj.xyz"

molRep2D, _ = loadData(12, xyz_traj_filename)
flatened_xyz = np.array(getFlattenedXyz(xyz_files_dir_name))




parent_dir_coulomb1D_to_xyz = pathlib.Path(os.path.abspath(os.path.dirname('__file__')) + "/data/testMolueculesMappedFromSampleSpace")
parent_dir_coulomb1D_to_xyz.mkdir(parents=True, exist_ok=True)

mapping_obj = MapCoulomb1dToXYZ(abs_path_xyz_dirname=str(parent_dir_coulomb1D_to_xyz), 
                                sorted_xyz_file="/home/panthibivek/thesis/GAN_pkg/data/exp/totalSortedTraj.txt", 
                                sorted_coulomb1D_file="/home/panthibivek/thesis/GAN_pkg/data/exp/totalSortedCoulomb1D.txt")
generated_xyz_list = mapping_obj.generateXYZ(coulomb1D_arr=molRep2D[:100], batch_size=1000)

2023-03-26 23:16:14.260534: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


input data size: (10000, 78)
output data size: (10000,)
77.83555460496353
69.49533210799979
73.19052683557553
83.00536123292925
76.55218145216872
72.99303885516493
42.6895877583383
66.82431109870664
4.668849344206756
65.6045247778846
70.2491074513061
81.91073428139727
77.33877036340618
66.88582988267409
67.23009112340617
71.12362567747023
79.19733110431939
37.2117699520335
70.2654522254371
36.18642071673528
65.84964327902752
72.29196818021882
98.01122331292866
62.52211371536948
79.24148528794403
75.97758719657698
75.94465598425984
69.29053590562643
36.59884636059263
66.20977673710635
67.90126463743404
49.161946785118595
68.44604027650401
63.29065137999219
71.48771937803225
41.43944943864149
73.7873754378089
49.26555765734307
66.46678505897643
72.49036953771432
79.03794617278986
32.00347713633147
39.20956699612725
72.10582866965434
42.71052851512924
67.54960648288362
43.45845518594927
43.68457617911102
36.9138606892385
71.1307953034854
59.47618611411041
77.83202545964072
39.396926702462

In [7]:
sq_diff = np.sum(np.sum((flatened_xyz[:100]-generated_xyz_list)**2))
print("square difference", sq_diff)
print("Deviation:", np.std(flatened_xyz[:100] - generated_xyz_list))
print("MSE: ", np.mean((flatened_xyz[:100] - generated_xyz_list)**2))

square difference 28.457044201293417
Deviation: 0.08758171494970564
MSE:  0.007904734500359282
